In [4]:
%%capture
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import nltk 
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw
from string import punctuation 

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')

data_url="https://raw.githubusercontent.com/liraop/recinfo_lab2/master/data/results.csv"
dados = pd.read_csv(data_url).replace(np.nan, '', regex=True)
n_documentos = dados.text.count()
palavras = []

### 1. Escolha uma estratégia de tokenização para a coleção que você está usando e justifique sua estratégia. É importante que você inclua decisões adicionais em relação ao que foi feito no Laboratório anterior (por exemplo, tratamento de maiúsculas/minúsculas e strings numéricas).

#### No lab anterior foi escolhida uma estratégia simples. Foram separadas as palavras consideradas úteis com uso da expressão regular `\w+`, colocando as palavras todas em caixa-baixa, sendo filtradas pelas condições de não serem *stopwords*, nem conunto de digitos numéricos, além de bigramas. Desta forma se anula boa parte do erro gerado por palavras não adequadas para análise. Contudo, ainda assim há perda de dados que seriam úteis para a análise.

#### Neste lab, será utilizado o módulo de *stemming* para português incluída na biblioteca NLTK, bem como os módulos de *stopwrods* e *tokenizer*. Na tokenização desta vez consideraremos datas no formato `aaaa` e também entre maiúsculas e minúsculas, adicionando mais complexidade na análise. Primeiro faremos a tokenização mais geral, separando as palavras, depois será feita a tokenização das datas.

In [2]:
word_token = rpt(r'\w+')

def tokenize_it(dataset, pattern, words):

    for artigo in dataset.text:
        tokens = []
        for token in pattern.tokenize(artigo.lower()):
        #se não é stopword e não é bigrama...
            if token not in stopwords and len(token) > 3 and not bool(re.search(r'\d', token)):
                tokens.append(token)
        words.extend(tokens)

### 2. Refaça a questão 2 do Laboratório anterior usando os tokens produzidos.

In [3]:
tokenize_it(dados, word_token, palavras)

df = pd.DataFrame(palavras, columns=['palavra'])
conta_palavras = df.palavra.value_counts().reset_index()
conta_palavras.columns = ['palavra', 'frequencia']
conta_palavras['r'] = conta_palavras.frequencia.rank(ascending=False, method='first')

acima_1000 = len(conta_palavras[conta_palavras.frequencia > 1000])
singulares = len(conta_palavras[conta_palavras.frequencia == 1])
total_itens = len(palavras)
vocabulario = len(set(palavras))

tabela = pd.DataFrame(data={"index": [  
             'Total documents', 
             'Total word occurences',
             'Vocabulary size',
             'Words occuring > 1000 times',
             'Words occuring once'
          ], "results": [
           n_documentos,
           total_itens,
           vocabulario,
           acima_1000,
           singulares
         ]})

tabela

,index,results
0,Total documents,249
1,Total word occurences,111362
2,Vocabulary size,21802
3,Words occuring > 1000 times,0
4,Words occuring once,10794


### 3. Aplique Stemming nos tokens produzidos e encontre 10 exemplos de falsos positivos e 10 exemplos de falsos negativos. Que impacto você acha que falsos positivos e negativos, como esses, teriam no processamento de consultas? Dê exemplos. 

### 4. Refaça a questão 3 do Laboratório anterior usando os tokens stemizados. Você percebeu alguma diferença em relação aos tokens sem stemming? Se sim, quais?